# KS-1: Clock Quadratic Test

**Goal & Pass Criterion**: Bures distance squared vs small θ is quadratic; DB²/θ² ~ constant for θ→0.

This notebook tests the fundamental assumption that quantum "clocks" exhibit quadratic distinguishability for small parameter changes. This is essential for the small-delta law that underlies capacity-time dilation effects.

## Setup

In [ ]:
import sys
sys.path.append('../src')

import numpy as np
import matplotlib.pyplot as plt
from capacity_time.clocks import run_all_ks1_tests, test_ks1_pure_state, test_ks1_noisy_state
from capacity_time.qfi import bures_small_delta, bures_distance_squared
from plotting import setup_figure, save_figure, plot_line_comparison

# Configure plotting
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 11

## Theory Background

For a quantum clock state |ψ(θ)⟩ evolving with parameter θ, the Bures distance squared should follow:

$$D_B^2(\theta) = 2(1 - \sqrt{F(\psi(0), \psi(\theta))})$$

For small θ, this should be quadratic: $D_B^2(\theta) \approx C\theta^2$ where C is a constant.

For our pointer state |+⟩ → Rz(θ)|+⟩, we expect C ≈ 0.25.

## Test 1: Pure Pointer State

In [ ]:
print("Testing pure pointer state |+⟩ → Rz(θ)|+⟩...")
print("=" * 50)

# Run pure state test
ratios_pure, mean_ratio_pure, pass_pure = test_ks1_pure_state(
    theta_max=0.2,
    n_points=20, 
    plot=True
)

print(f"\nResult: {'PASS' if pass_pure else 'FAIL'}")
print(f"Mean ratio DB²/θ²: {mean_ratio_pure:.4f}")
print(f"Expected theoretical value: ~0.25")

## Test 2: Noisy Pointer State

In [ ]:
print("Testing noisy pointer state with T1=50μs, T2=30μs...")
print("=" * 50)

# Run noisy state test
ratios_noisy, mean_ratio_noisy, pass_noisy = test_ks1_noisy_state(
    theta_max=0.2,
    n_points=20,
    T1=50e-6,  # 50 μs
    T2=30e-6,  # 30 μs
    plot=True
)

print(f"\nResult: {'PASS' if pass_noisy else 'FAIL'}")
print(f"Mean ratio DB²/θ²: {mean_ratio_noisy:.4f}")
print(f"Note: Noise reduces ratio but quadratic behavior should persist")

## Comparison Analysis

In [ ]:
# Compare pure vs noisy ratios
theta_vals = np.linspace(0.01, 0.2, 20)

ratio_comparison = {
    'Pure State': ratios_pure,
    'Noisy State': ratios_noisy
}

fig = plot_line_comparison(
    theta_vals, ratio_comparison,
    title="KS-1: Quadratic Ratio Comparison", 
    xlabel="θ (radians)",
    ylabel="DB²/θ²",
    figsize='double',
    save_path='figures/KS1_ratio_comparison'
)

plt.axhline(0.25, color='red', linestyle=':', alpha=0.7, label='Theoretical (pure)')
plt.axhline(mean_ratio_pure, color='blue', linestyle='--', alpha=0.7, 
           label=f'Mean pure: {mean_ratio_pure:.3f}')
plt.axhline(mean_ratio_noisy, color='orange', linestyle='--', alpha=0.7,
           label=f'Mean noisy: {mean_ratio_noisy:.3f}')
plt.legend()
plt.show()

## Custom Angle Range Test

In [ ]:
# Test even smaller angles to verify θ→0 limit
print("Testing very small angles (θ < 0.1 rad)...")

from capacity_time.clocks import pointer_state

small_angles = np.linspace(0.005, 0.1, 15)
theta_arr, DB2_arr, ratio_arr = bures_small_delta(
    small_angles.tolist(),
    lambda theta: pointer_state(theta)
)

# Plot results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Linear relationship DB² vs θ²  
ax1.scatter(theta_arr**2, DB2_arr, alpha=0.7, s=50)
ax1.plot(theta_arr**2, 0.25 * theta_arr**2, 'r--', label='Theoretical: 0.25θ²')
ax1.set_xlabel('θ² (rad²)')
ax1.set_ylabel('DB²')
ax1.set_title('Very Small Angles: DB² vs θ²')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Ratio consistency
ax2.plot(theta_arr, ratio_arr, 'bo-', markersize=4)
ax2.axhline(0.25, color='red', linestyle='--', alpha=0.7, label='Theoretical: 0.25')
ax2.axhline(np.mean(ratio_arr), color='green', linestyle=':', alpha=0.7,
           label=f'Mean: {np.mean(ratio_arr):.3f}')
ax2.set_xlabel('θ (rad)')
ax2.set_ylabel('DB²/θ²')
ax2.set_title('Small Angle Limit Test')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.suptitle('KS-1: Ultra-Small Angle Verification')
plt.tight_layout()
plt.savefig('figures/KS1_small_angle_limit.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"Very small angle mean ratio: {np.mean(ratio_arr):.4f}")
print(f"Standard deviation: {np.std(ratio_arr):.4f}")
print(f"Relative variation: {np.std(ratio_arr)/np.mean(ratio_arr)*100:.1f}%")

## Final KS-1 Verdict

In [ ]:
# Overall KS-1 assessment
overall_pass = pass_pure and pass_noisy

print("=" * 60)
print("KS-1: CLOCK QUADRATIC TEST - FINAL RESULTS")
print("=" * 60)
print(f"Pure state test: {'PASS' if pass_pure else 'FAIL'}")
print(f"Noisy state test: {'PASS' if pass_noisy else 'FAIL'}")
print(f"")
print(f"Expected behavior: DB²/θ² ≈ 0.25 (constant for small θ)")
print(f"Pure state result: {mean_ratio_pure:.4f}")
print(f"Noisy state result: {mean_ratio_noisy:.4f}")
print(f"")
print(f"KS-1 OVERALL: {'PASS' if overall_pass else 'FAIL'}")
print("=" * 60)

if overall_pass:
    print("✓ Quadratic small-delta law confirmed")
    print("✓ Ready for KS-2 motion vs demand tests")
else:
    print("✗ Quadratic behavior not established")
    print("✗ May need to revise theoretical foundations")

## Summary

KS-1 tests the fundamental quantum clock assumption that enables all subsequent capacity-time dilation experiments. The quadratic small-delta law DB²(θ) ∝ θ² is essential because:

1. It establishes quantum Fisher Information scaling
2. It provides the foundation for demand estimation via variance
3. It ensures that small changes in evolution rate create detectable effects

**Pass Criteria**: 
- DB²/θ² ratio should be approximately constant for small θ
- Pure state ratio should be close to theoretical value (~0.25)
- Noisy state should maintain quadratic behavior (possibly with reduced ratio)

If KS-1 fails, the entire capacity-time dilation framework needs theoretical revision.